# Compilación

En este reporte se estarán analizando y desarrollando las diferentes partes del transpilador del lenguaje RoadToCivilization al lenguaje python, entre ellas están, el lexer, la gramática, el parser, el análisis semántico y el transpilado a python.

## Lexer


